In [3]:
import json
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

def load_json_file(filename):
    """Carga un archivo JSON y maneja posibles errores."""
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filename}")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {filename} no es un JSON válido")
        return None
    except Exception as e:
        print(f"Error inesperado al cargar {filename}: {str(e)}")
        return None

def get_demand_data(profesores_input):
    """Analiza la demanda agrupando asignaturas por cantidad de vacantes."""
    # Extraer todas las asignaturas
    asignaturas = []
    for profesor in profesores_input:
        for asignatura in profesor['Asignaturas']:
            asignaturas.append({
                'codigo': asignatura['CodigoAsignatura'],
                'nombre': asignatura['Nombre'],
                'vacantes': asignatura['Vacantes'],
                'horas': asignatura['Horas']
            })
    
    # Agrupar por cantidad de vacantes
    grupos_vacantes = defaultdict(list)
    for asignatura in asignaturas:
        grupos_vacantes[asignatura['vacantes']].append(asignatura)
    
    # Crear datos agregados
    demand_data = []
    for vacantes, grupo in grupos_vacantes.items():
        demand_data.append({
            'vacantes': vacantes,
            'horas_totales': sum(a['horas'] for a in grupo),
            'cantidad_asignaturas': len(grupo),
            'asignaturas': [a['codigo'] for a in grupo]
        })
    
    return sorted(demand_data, key=lambda x: x['vacantes'], reverse=True)

def get_capacity_data(salas_input):
    """Analiza la capacidad agrupando salas por capacidad."""
    # Agrupar salas por capacidad
    grupos_capacidad = defaultdict(list)
    for sala in salas_input:
        grupos_capacidad[sala['Capacidad']].append(sala)
    
    # Crear datos agregados (45 bloques disponibles por sala)
    capacity_data = []
    for capacidad, grupo in grupos_capacidad.items():
        capacity_data.append({
            'capacidad': capacidad,
            'bloques_disponibles': len(grupo) * 45,  # 45 bloques por sala
            'cantidad_salas': len(grupo),
            'salas': [s['Codigo'] for s in grupo]
        })
    
    return sorted(capacity_data, key=lambda x: x['capacidad'], reverse=True)

def analyze_gaps(demand_data, capacity_data):
    """Analiza las brechas entre demanda y capacidad disponible."""
    gap_analysis = []
    
    for demand in demand_data:
        # Encontrar todas las salas que pueden acomodar esta cantidad de vacantes
        salas_disponibles = [c for c in capacity_data if c['capacidad'] >= demand['vacantes']]
        bloques_disponibles = sum(s['bloques_disponibles'] for s in salas_disponibles)
        
        gap_analysis.append({
            'vacantes': demand['vacantes'],
            'horas_requeridas': demand['horas_totales'],
            'bloques_disponibles': bloques_disponibles,
            'deficit': max(0, demand['horas_totales'] - bloques_disponibles),
            'cantidad_asignaturas': demand['cantidad_asignaturas']
        })
    
    return gap_analysis

def create_summary_tables(demand_data, capacity_data, gap_analysis):
    """Crea las tablas de resumen del análisis."""
    # Tabla de demanda
    df_demand = pd.DataFrame(demand_data)
    df_demand = df_demand.drop('asignaturas', axis=1)
    
    # Tabla de capacidad
    df_capacity = pd.DataFrame(capacity_data)
    df_capacity = df_capacity.drop('salas', axis=1)
    
    # Tabla de brechas
    df_gaps = pd.DataFrame(gap_analysis)
    
    return df_demand, df_capacity, df_gaps

def create_visualizations(gap_analysis):
    """Crea visualizaciones de demanda vs capacidad y demanda específica."""
    data = pd.DataFrame(gap_analysis)
    
    # 1. Gráfico comparativo de demanda vs capacidad
    plt.figure(figsize=(12, 6))
    plt.bar(range(len(data)), data['horas_requeridas'], 
           label='Horas Requeridas', alpha=0.8)
    plt.bar(range(len(data)), data['bloques_disponibles'], 
           label='Bloques Disponibles', alpha=0.6)
    
    plt.xlabel('Grupos por Vacantes')
    plt.ylabel('Horas/Bloques')
    plt.title('Demanda vs Capacidad por Grupo de Vacantes')
    plt.legend()
    plt.xticks(range(len(data)), data['vacantes'], rotation=45)
    plt.tight_layout()
    plt.savefig('demanda_vs_capacidad.png')
    plt.close()
    
    # 2. Gráfico específico de demanda
    plt.figure(figsize=(12, 6))
    bars = plt.bar(range(len(data)), data['horas_requeridas'], 
            color='darkblue', alpha=0.7)
    
    # Añadir etiquetas de valor encima de cada barra
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(height)}',
                ha='center', va='bottom')
    
    plt.xlabel('Grupos por Vacantes')
    plt.ylabel('Horas Requeridas')
    plt.title('Demanda de Horas por Grupo de Vacantes')
    
    # Ajustar etiquetas del eje x para incluir vacantes y cantidad de asignaturas
    plt.xticks(range(len(data)), 
               [f'Vac:{v}' for v, a in zip(data['vacantes'], data['cantidad_asignaturas'])], 
               rotation=45)
    
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig('demanda_especifica.png')
    plt.close()

def save_results(df_demand, df_capacity, df_gaps):
    """Guarda los resultados en archivos CSV."""
    df_demand.to_csv('demanda.csv', index=False)
    df_capacity.to_csv('capacidad.csv', index=False)
    df_gaps.to_csv('brechas.csv', index=False)

def main():
    # 1. Cargar datos
    profesores_input = load_json_file('../../agent_input/InputOfProfesores.json')
    salas_input = load_json_file('../../agent_input/InputOfSala.json')
    
    if profesores_input is None or salas_input is None:
        return
    
    # 2. Analizar demanda
    demand_data = get_demand_data(profesores_input)
    print("\nAnálisis de Demanda:")
    print(pd.DataFrame(demand_data).drop('asignaturas', axis=1).to_string())
    
    # 3. Analizar capacidad
    capacity_data = get_capacity_data(salas_input)
    print("\nAnálisis de Capacidad:")
    print(pd.DataFrame(capacity_data).drop('salas', axis=1).to_string())
    
    # 4. Analizar brechas
    gap_analysis = analyze_gaps(demand_data, capacity_data)
    print("\nAnálisis de Brechas:")
    print(pd.DataFrame(gap_analysis).to_string())
    
    # 5. Crear tablas de resumen
    df_demand, df_capacity, df_gaps = create_summary_tables(demand_data, capacity_data, gap_analysis)
    
    # 6. Crear visualizaciones
    create_visualizations(gap_analysis)
    
    # 7. Guardar resultados
    save_results(df_demand, df_capacity, df_gaps)
    
    print("\nAnálisis completado. Se han guardado los resultados en archivos CSV y la visualización en 'demanda_vs_capacidad.png'")
    
    # 8. Identificar conflictos críticos
    conflictos = df_gaps[df_gaps['deficit'] > 0].sort_values('deficit', ascending=False)
    if not conflictos.empty:
        print("\nConflictos Críticos Identificados:")
        print("Grupos con déficit de bloques disponibles:")
        print(conflictos.to_string())
        
        total_deficit = conflictos['deficit'].sum()
        print(f"\nDéficit total de bloques: {total_deficit}")
        print(f"Número de grupos afectados: {len(conflictos)}")
        print(f"Total de asignaturas afectadas: {conflictos['cantidad_asignaturas'].sum()}")

if __name__ == "__main__":
    main()


Análisis de Demanda:
    vacantes  horas_totales  cantidad_asignaturas
0         67              5                     2
1         63             10                     4
2         61              5                     2
3         53              6                     2
4         49              8                     4
5         48              9                     4
6         47             10                     4
7         46             20                     5
8         45              6                     4
9         44              5                     2
10        43             15                     6
11        42             12                     1
12        41             42                     4
13        40             20                     5
14        39             26                     7
15        38            134                    13
16        37            115                    13
17        36             23                    10
18        35             17 